In [1]:
import os, glob, pickle
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"]='2,3'
os.environ["CUDA_VISIBLE_DEVICES"]='3'
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
train_ci_ids = ["HM0001", "HM0002", "HS0003", "HS0004", "HS0005", "HS0006", "HS0011"]
# test_ci_ids = ["HM0004", "HM0005", "HS0001", "HS0002", "HS0009", "HS0010", "HS0012","HS0013"]

train_hi_ids = ["HL0052", "HL0074", "HL0028", "HL0150", "HL0039", "HL0019", "HL0029"]
# test_hi_ids = ["HL0005", "HL0075", "HL0024", "HL0034", "HL0089"]


def loads_npy_with_y(base_path, ids, y_value):
    loaded_npa_list = []
    for id in ids:
        file_path = glob.glob(f'{base_path}/{id}*.npy')[0]
        if not isinstance(file_path, str):
            raise Exception(f"could'n find id({id}) file ")
        npa = np.load(file_path)
        n = npa.shape[0]
        y = np.array([y_value] * n, dtype=npa.dtype)
        npa = np.concatenate([npa,y.reshape(-1,1)], axis=1)
        loaded_npa_list.append(npa)
    
    return np.concatenate(loaded_npa_list, axis=0)


def norm(X):
    ## normalized to have zero mean and unit variance
    
    raise NotImplementedError
#     return [(x - np.mean(x)) / np.sqrt(np.var(x) + 1e-5) for x in input_values]


ci_with_y = loads_npy_with_y('CI', train_ci_ids, 1)
print("ci_with_y'shape: ", ci_with_y.shape)

hi_with_y = loads_npy_with_y('HI', train_hi_ids, 0)
print("hi_with_y'shape: ", hi_with_y.shape)

x_with_y = np.concatenate([ci_with_y, hi_with_y], axis=0)
np.random.shuffle(x_with_y)


X = x_with_y[:,:-1]
Y = x_with_y[:,-1]
print("X shape: ", X.shape, "Y shape: ", Y.shape)


validation_rate = 0.25
h = X.shape[0]
val_num = int(h * validation_rate)
train_num = h - val_num

train_num = (train_num//100)*100
val_num = (val_num//100)*100

train_X = X[:train_num]
train_Y = Y[:train_num]
print("train_X'shape: ", train_X.shape)
val_X = X[-val_num:]
val_Y = Y[-val_num:]
print("val_X'shape: ", val_X.shape)

ci_with_y'shape:  (31345, 64001)
hi_with_y'shape:  (48036, 64001)
X shape:  (79381, 64000) Y shape:  (79381,)
train_X'shape:  (59500, 64000)
val_X'shape:  (19800, 64000)


In [3]:
class MHAttention(tf.keras.layers.Layer):
    def __init__(self, d_model:int, num_heads:int, batch_size:int=None):
#     def __init__(self, d_model:int, num_heads:int):
        super(MHAttention,self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = self.d_model // self.num_heads
        self.batch_size = batch_size
        assert self.d_model % self.num_heads == 0

        self.sqrt_depth = tf.cast(self.depth, dtype=tf.float32)

        self.query = tf.keras.layers.Dense(self.d_model)
        self.key = tf.keras.layers.Dense(self.d_model)
        self.value = tf.keras.layers.Dense(self.d_model)
        self.outweight = tf.keras.layers.Dense(self.d_model)
        self.softmax = tf.keras.layers.Softmax(axis=-1)


    def split_heads(self, input):
        if self.batch_size == None:
            batch_size = tf.shape(input)[0]
        else: 
            batch_size = self.batch_size
        input = tf.reshape(input,(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(input, perm=[0,2,1,3])


    def __call__(self, input):
        if self.batch_size == None:
            batch_size = tf.shape(input)[0]
        else: 
            batch_size = self.batch_size
        query = self.query(input)
        key = self.key(input)
        value = self.value(input)

        query_splitted = self.split_heads(query)
        key_splitted = self.split_heads(key)
        value_splitted = self.split_heads(value)

        q_mat_k = tf.matmul(query_splitted, key_splitted, transpose_b=True)
        q_mat_k = q_mat_k / self.sqrt_depth

        q_mat_k_soft = self.softmax(q_mat_k)
        attention_score = tf.matmul(q_mat_k_soft, value_splitted)
        attention_score = tf.transpose(attention_score, perm=[0,2,1,3])
        attention_score = tf.reshape(attention_score, (batch_size, -1, self.d_model))

        return self.outweight(attention_score)


class MyEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model:int, num_heads:int, batch_size:int=None):
        super(MyEncoder,self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.multi_head_attention = MHAttention(self.d_model, self.num_heads, self.batch_size)

        self.dense1 = tf.keras.layers.Dense(d_model)
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.act1 = tf.keras.layers.Activation('relu')

    def __call__(self, input):
        a = self.multi_head_attention(input)
        con = tf.concat([input,a], axis=-1)
        o1 = self.dense1(con)
        o1 = self.layer_norm(o1)
        o1 = self.act1(o1)       

        return o1

class Res1(tf.keras.layers.Layer):
# class Res1(tf.keras.Model):
    def __init__(self,filters:int,kernel_size:int,padding:str,activation:str, flag_res:bool=True):
        super(Res1,self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.FLAG_RES = flag_res

        self.conv1 = Conv1D(self.filters, kernel_size=self.kernel_size, padding=self.padding)
        self.batch1 = BatchNormalization()
        self.act1 = Activation(self.activation)

        self.conv2 = Conv1D(self.filters, kernel_size=self.kernel_size, padding=self.padding)
        self.batch2 = BatchNormalization()
        self.act2 = Activation(self.activation)

        self.conv3 = Conv1D(self.filters, kernel_size=self.kernel_size, padding=self.padding)
        self.batch3 = BatchNormalization()
        self.act3 = Activation(self.activation)

        self.pool = MaxPool1D(strides=2)


    def __call__(self, input):
        x1 = self.conv1(input)
        x2 = self.batch1(x1)
        x3 = self.act1(x2)

        x4 = self.conv2(x3)
        x5 = self.batch2(x4)
        x6 = self.act2(x5)

        x7 = self.conv3(x6)
        x8 = self.batch3(x7)
        x9 = self.act3(x8)

        if self.FLAG_RES:
            x_added = tf.add(x9, x3)
            return self.pool(x_added)
        else :
            return self.pool(x9)


        
class MobileDense1(tf.keras.layers.Layer):
    ## the number of filters must be twice of input channels because the output will be concatenated. 
    def __init__(self,filters:int, kernel_size:int=3, padding:str='same', activation:str='relu', depth_mul:int=4):
        super(MobileDense1,self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.depth_mul = depth_mul

        if self.filters & 1:
            raise Exception("from MobileDense1: the parameter named 'filters' must be even number")

        self.conv1 = DepthwiseConv2D(self.kernel_size, depth_multiplier=self.depth_mul, padding=self.padding)
        self.bn1 = BatchNormalization()
        self.act1 = Activation(self.activation)
        self.conv2 = Conv2D(self.filters//2, 1, padding=self.padding)
        self.bn2 = BatchNormalization()
        self.act2 = Activation(self.activation)
        self.pool = MaxPool2D(strides=2)
        
    def __call__(self, input):
        if input.shape[-1] != self.filters//2:
            raise Exception("from MobileDense1: the input channels must be half of the 'filters' parameter")

        x1 = self.conv1(input)
        x2 = self.bn1(x1)
        x3 = self.act1(x2)
        x4 = self.conv2(x3)
        x5 = self.bn2(x4)
        x6 = self.act2(x5)
        x7 = tf.concat([input, x6],axis=-1)
        
        return self.pool(x7)

        
        

In [4]:
def make_model1():
    ## basic CNN
    ACTIVATION_FN = 'elu'
    filters = 32

    input_layer = Input(shape=(64000,1))
    conv1 = Conv1D(filters, kernel_size=3, padding='same')(input_layer)
    batch1 = BatchNormalization()(conv1)
    act1 = Activation(ACTIVATION_FN)(batch1)
    pool1 = MaxPool1D(strides=2)(act1)

    ## 32000, 64
    conv2 = Conv1D(filters *2, kernel_size=3, padding='same')(pool1)
    batch2 = BatchNormalization()(conv2)
    act2 = Activation(ACTIVATION_FN)(batch2)
    pool2 = MaxPool1D(strides=2)(act2)

    ## 16000, 128
    conv3 = Conv1D(filters *4, kernel_size=3, padding='same')(pool2)
    batch3 = BatchNormalization()(conv3)
    act3 = Activation(ACTIVATION_FN)(batch3)
    pool3 = MaxPool1D(strides=2)(act3)

    ## 8000, 256
    conv4 = Conv1D(filters*8, kernel_size=3, padding='same')(pool3)
    batch4 = BatchNormalization()(conv4)
    act4 = Activation(ACTIVATION_FN)(batch4)
    pool4 = MaxPool1D(strides=2)(act4)

    ## 4000, 512
    conv5 = Conv1D(filters *16, kernel_size=3, padding='same')(pool4)
    batch5 = BatchNormalization()(conv5)
    act5 = Activation(ACTIVATION_FN)(batch5)
    pool5 = MaxPool1D(strides=2)(act5)

    ## 2000, 512
    conv6 = Conv1D(filters *16, kernel_size=3, padding='same')(pool5)
    batch6 = BatchNormalization()(conv6)
    flat1 = Flatten()(batch6)
    dense1 = Dense(200, activation='relu')(flat1)
    batch7 = BatchNormalization()(dense1)
    dense2 = Dense(20, activation='relu')(batch7)
    # dense2 = Dense(20, activation='relu')(dense1)
    output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(input_layer, output_layer)

#     model.summary()
    return model

In [5]:
def make_model2(batch_size):
    ## resnet only
    BATCH_SIZE = batch_size
    ACTIVATION_FN = 'elu'
    filters = 32


#     input_layer = Input(shape=(64000,1), batch_size=BATCH_SIZE)
    input_layer = Input(shape=(64000,1))

    res1 = Res1(filters, 3, 'same', ACTIVATION_FN)(input_layer)
    ## 32000, 32

    res2 = Res1(filters*2, 3, 'same', ACTIVATION_FN)(res1)
    ## 16000, 64

    res3 = Res1(filters*4, 3, 'same', ACTIVATION_FN)(res2)
    ## 8000, 128

    res4 = Res1(filters*8, 3, 'same', ACTIVATION_FN)(res3)
    ## 4000, 256

    res5 = Res1(filters*16, 3, 'same', ACTIVATION_FN)(res4)
    ## 2000, 512

    res6 = Res1(filters*16, 3, 'same', ACTIVATION_FN)(res5)
    ## 1000, 512
    
    res7 = Res1(filters*16, 3, 'same', ACTIVATION_FN)(res6)
    ## 500, 512
    
    flat5 = Flatten()(res7)
#     flat5 = Flatten()(res1)
    dense5 = Dense(200, activation='relu')(flat5)
    batch5 = BatchNormalization()(dense5)
    dense6 = Dense(20, activation='relu')(batch5)
    output_layer = Dense(1, activation='sigmoid')(dense6)    
    
    return Model(input_layer, output_layer)
    

In [6]:
def make_model3(batch_size):
    ## CNN + ATTENTION
    ACTIVATION_FN = 'elu'
    filters = 32

    input_layer = Input(shape=(64000,1))
    conv1 = Conv1D(filters, kernel_size=3, padding='same')(input_layer)
    batch1 = BatchNormalization()(conv1)
    act1 = Activation(ACTIVATION_FN)(batch1)
    pool1 = MaxPool1D(strides=2)(act1)

    ## 32000, 64
    conv2 = Conv1D(filters *2, kernel_size=3, padding='same')(pool1)
    batch2 = BatchNormalization()(conv2)
    act2 = Activation(ACTIVATION_FN)(batch2)
    pool2 = MaxPool1D(strides=2)(act2)

    ## 16000, 128
    conv3 = Conv1D(filters *4, kernel_size=3, padding='same')(pool2)
    batch3 = BatchNormalization()(conv3)
    act3 = Activation(ACTIVATION_FN)(batch3)
    pool3 = MaxPool1D(strides=2)(act3)

    ## 8000, 256
    conv4 = Conv1D(filters*8, kernel_size=3, padding='same')(pool3)
    batch4 = BatchNormalization()(conv4)
    act4 = Activation(ACTIVATION_FN)(batch4)
    pool4 = MaxPool1D(strides=2)(act4)

    ## 4000, 512
    conv5 = Conv1D(filters *16, kernel_size=3, padding='same')(pool4)
    batch5 = BatchNormalization()(conv5)
    act5 = Activation(ACTIVATION_FN)(batch5)
    pool5 = MaxPool1D(strides=2)(act5)

    ## 2000, 512
    conv6 = Conv1D(filters *16, kernel_size=3, padding='same')(pool5)
    tr1 = tf.transpose(conv6, perm=[0,2,1])
    enc1 = MyEncoder(2000,8,tf.shape(conv1)[0])(tr1)
    
    
    flat1 = Flatten()(enc1)
    dense1 = Dense(200, activation='relu')(flat1)
    batch7 = BatchNormalization()(dense1)
    dense2 = Dense(20, activation='relu')(batch7)
    # dense2 = Dense(20, activation='relu')(dense1)
    output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(input_layer, output_layer)

#     model.summary()
    return model

#     conv1 = Conv1D(filters *16, kernel_size=3, padding='same')(res5)
#     ## 2000, 512
#     tr1 = tf.transpose(conv1,perm=[0,2,1])
#     # print("input_layer, ", res10)
#     enc1 = MyEncoder(2000,8,BATCH_SIZE)(tr1)

#     flat1 = Flatten()(enc1)
#     batch6 = BatchNormalization()(flat1)
#     dense1 = Dense(200, activation='relu')(batch6)
#     batch7 = BatchNormalization()(dense1)
#     dense2 = Dense(20, activation='relu')(batch7)
#     # dense2 = Dense(20, activation='relu')(dense1)
#     output_layer = Dense(1, activation='sigmoid')(dense2)

#     model = Model(input_layer, output_layer)
    
#     return model


In [7]:
def make_model4(batch_size):
    ## RESNET + ATTENTION
    BATCH_SIZE = batch_size
    ACTIVATION_FN = 'elu'
    filters = 32


    input_layer = Input(shape=(64000,1))

    res1 = Res1(filters, 3, 'same', ACTIVATION_FN)(input_layer)
    ## 32000, 32

    res2 = Res1(filters*2, 3, 'same', ACTIVATION_FN)(res1)
    ## 16000, 64

    res3 = Res1(filters*4, 3, 'same', ACTIVATION_FN)(res2)
    ## 8000, 128

    res4 = Res1(filters*8, 3, 'same', ACTIVATION_FN)(res3)
    ## 4000, 256

    res5 = Res1(filters*16, 3, 'same', ACTIVATION_FN)(res4)
    ## 2000, 512

    res6 = Res1(filters*8, 3, 'same', ACTIVATION_FN)(res5)
    ## 1000, 256

    
    conv1 = Conv1D(filters*4, kernel_size=3, padding='same')(res6)
    ## 1000, 128
    tr1 = tf.transpose(conv1,perm=[0,2,1])
    # print("input_layer, ", res10)
    enc1 = MyEncoder(1000,10,tf.shape(res1)[0])(tr1)
    
    flat1 = Flatten()(enc1)
#     batch6 = BatchNormalization()(flat1)
#     dense1 = Dense(200, activation='relu')(batch6)
    dense1 = Dense(200, activation='relu')(flat1)
    batch7 = BatchNormalization()(dense1)
    dense2 = Dense(20, activation='relu')(batch7)
    # dense2 = Dense(20, activation='relu')(dense1)
    output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(input_layer, output_layer)
    
    return model


In [13]:
def make_model5(batch_size):
    ## mobile + dense + att
    BATCH_SIZE = batch_size
    ACTIVATION_FN = 'elu'
    filters = 32
    
    input_layer = Input(shape=(64000,1))
    conv = Conv1D(32,3, padding='same')(input_layer)
    bn = BatchNormalization()(conv)
    act = Activation('relu')(bn)
    reshaped1 = tf.reshape(act, (-1,320,200,32))
    
    md1 = MobileDense1(64, activation=ACTIVATION_FN)(reshaped1)
    ## 160,100, 64

    md2 = MobileDense1(128, activation=ACTIVATION_FN)(md1)
    ## 80, 50, 128

    md3 = MobileDense1(256, activation=ACTIVATION_FN)(md2)
    ## 40, 25, 256

    md4 = MobileDense1(512, activation=ACTIVATION_FN)(md3)
    ## 20, 12, 128

#     md5 = MobileDense1(1024, activation=ACTIVATION_FN)(md4)
#     ## 25,1024
    
    conv1 = Conv2D(10,kernel_size = 1, padding='same')(md4)
    flat1 = Flatten()(conv1)
    dense1 = Dense(200, activation='relu')(flat1)
    bn2 = BatchNormalization()(dense1)
    dense2 = Dense(20, activation='relu')(bn2)
    output_layer = Dense(1, activation='sigmoid')(dense2)
    
#     reshaped2 = tf.reshape(md4,(-1,2500,512))
#     tr1 = tf.transpose(reshaped2,[0,2,1])
#     enc1 = MyEncoder(2500,10,tf.shape(conv)[0])(tr1)
#     enc2 = MyEncoder(2500,10,tf.shape(conv)[0])(enc1)

#     flat1 = Flatten()(enc2)
#     batch6 = BatchNormalization()(flat1)
#     dense1 = Dense(200, activation='relu')(batch6)
#     batch7 = BatchNormalization()(dense1)
#     dense2 = Dense(20, activation='relu')(batch7)
#     # dense2 = Dense(20, activation='relu')(dense1)
#     output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(input_layer, output_layer)
    
    return model

In [14]:
EPOCHS = 10
BATCH_SIZE = 10
MODEL_NUM = 5

def make_model_with_num(MODEL_NUM, BATCH_SIZE):
    if MODEL_NUM == 1:
        model = make_model1()
    elif MODEL_NUM == 2:
        model = make_model2(BATCH_SIZE)
    elif MODEL_NUM == 3:
        model = make_model3(BATCH_SIZE)
    elif MODEL_NUM == 4:
        model = make_model4(BATCH_SIZE)
    elif MODEL_NUM == 5:
        model = make_model5(BATCH_SIZE)
    
    return model
    
model = make_model_with_num(MODEL_NUM,BATCH_SIZE)
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64000, 1)]   0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 64000, 32)    128         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 64000, 32)    128         conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 64000, 32)    0           batch_normalization_10[0][0]     
____________________________________________________________________________________________

In [15]:
def training_with_small_dataset(train_X, train_Y, val_X, val_Y, BATCH_SIZE, EPOCHS):

#     LIMIT_N = train_X.shape[0]
    SPLIT_N = 11
    devided_train_index = list(map(int,np.linspace(0,train_X.shape[0],SPLIT_N)))
    devided_val_index = list(map(int,np.linspace(0,val_X.shape[0],SPLIT_N)))
    
    print('devided_train_index: ',devided_train_index)
    print('devided_val_index: ',devided_val_index)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=2,
        verbose=1,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    )
    
    
    losses = np.zeros((SPLIT_N-1,EPOCHS))
    val_losses = np.zeros((SPLIT_N-1,EPOCHS))
    accs = np.zeros((SPLIT_N-1,EPOCHS))
    val_accs = np.zeros((SPLIT_N-1,EPOCHS))
    
    for i in range(SPLIT_N-1):
#     i=0
#         start_num = devided_index[i]
#     start_num = 0
#         end_num = devided_index[i+1]
#     end_num = 2000
        print(f"sub_dataset:{i}")
        history = model.fit(train_X[ devided_train_index[i]:devided_train_index[i+1] ], train_Y[ devided_train_index[i]:devided_train_index[i+1] ], validation_data=(val_X[ devided_val_index[i]:devided_val_index[i+1] ], val_Y[ devided_val_index[i]:devided_val_index[i+1] ]), batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True, verbose=2, callbacks=[early_stopping])
        
        t_losses = history.history['loss']
        losses[i,:len(t_losses)]= t_losses
        
        t_accs = history.history['accuracy']
        accs[i,:len(t_accs)]= t_accs
        
        t_val_losses = history.history['val_loss']
        val_losses[i, :len(t_val_losses)] = t_val_losses
        
        t_val_accs = history.history['val_accuracy']
        val_accs[i, :len(t_val_accs)] = t_val_accs
        
        

    
    return {"accuracy": accs, "val_accuracy": val_accs, "loss": losses, "val_loss": val_losses}

In [16]:
def training_all_at_once(train_X, train_Y, val_X, val_Y, BATCH_SIZE, EPOCHS):
    history = model.fit(train_X,train_Y, validation_data=(val_X, val_Y), batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True, verbose=2)
    
    return history.history
    

In [17]:
results = training_with_small_dataset(train_X, train_Y, val_X, val_Y, BATCH_SIZE, EPOCHS)
# results = training_all_at_once(train_X, train_Y, val_X, val_Y, BATCH_SIZE, EPOCHS)



devided_train_index:  [0, 5950, 11900, 17850, 23800, 29750, 35700, 41650, 47600, 53550, 59500]
devided_val_index:  [0, 1980, 3960, 5940, 7920, 9900, 11880, 13860, 15840, 17820, 19800]
sub_dataset:0
Epoch 1/10
595/595 - 116s - loss: 0.3085 - accuracy: 0.8647 - val_loss: 2.1547 - val_accuracy: 0.4141
Epoch 2/10
595/595 - 103s - loss: 0.1019 - accuracy: 0.9634 - val_loss: 0.0492 - val_accuracy: 0.9828
Epoch 3/10
595/595 - 104s - loss: 0.0549 - accuracy: 0.9822 - val_loss: 0.3050 - val_accuracy: 0.8737
Epoch 4/10
595/595 - 105s - loss: 0.0521 - accuracy: 0.9825 - val_loss: 0.1522 - val_accuracy: 0.9389
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping
sub_dataset:1
Epoch 1/10
595/595 - 110s - loss: 0.0759 - accuracy: 0.9721 - val_loss: 4.4493 - val_accuracy: 0.6323
Epoch 2/10
595/595 - 112s - loss: 0.0490 - accuracy: 0.9825 - val_loss: 0.0644 - val_accuracy: 0.9768
Epoch 3/10
595/595 - 109s - loss: 0.0437 - accuracy: 0.9854 - val_loss: 0.2868 - val_accurac

In [18]:
## Save model
# model.save(f"model{MODEL_NUM}_E{EPOCHS}_B{BATCH_SIZE}_T{train_num}_V{val_num}_val-acc{round(results['val_accuracy'][-1],3)}.h5")
# model.save(f"model{MODEL_NUM}_E{EPOCHS}_B{BATCH_SIZE}_T{train_num}_V{val_num}_val-acc{round(np.mean(results['val_accuracy'][:,-1]),3)}.h5")

## Save history.history
# np.save(f"model{MODEL_NUM}_E{EPOCHS}_B{BATCH_SIZE}_T{train_num}_V{val_num}_val-acc{round(np.mean(results['val_accuracy'][:,-1]),3)}_history.npy", results)
# a = np.load("model4_E10_B10_T59500_V19800_val-acc0.0_history.npy", allow_pickle=True)
# print(a)

evaluated = model.evaluate(val_X, val_Y)
print("val_loss, val_acc: ",evaluated)
val_loss, val_acc = evaluated

model.save(f"model{MODEL_NUM}_E{EPOCHS}_B{BATCH_SIZE}_T{train_num}_V{val_num}_val-acc{round(val_acc,3)}.h5")
np.save(f"model{MODEL_NUM}_E{EPOCHS}_B{BATCH_SIZE}_T{train_num}_V{val_num}_val-acc{round(val_acc,3)}_history.npy", results)

619/619 [==============================] - 43s 64ms/step - loss: 0.0204 - accuracy: 0.9949
val_loss, val_acc:  [0.02040841057896614, 0.9948989748954773]


In [ ]:
## model2_E15_B10_N2000_val-acc0.564.h5
## 트랜스포머없이 resnet 만 사용결과
## 데이터 2000개만 (메모리오류)

# Epoch 1/15
# 200/200 - 65s - loss: 0.3049 - accuracy: 0.8770 - val_loss: 2.3435 - val_accuracy: 0.5420
# Epoch 2/15
# 200/200 - 43s - loss: 0.1117 - accuracy: 0.9610 - val_loss: 0.3934 - val_accuracy: 0.8285
# Epoch 3/15
# 200/200 - 43s - loss: 0.0629 - accuracy: 0.9815 - val_loss: 7.9869 - val_accuracy: 0.4540
# Epoch 4/15
# 200/200 - 43s - loss: 0.0314 - accuracy: 0.9920 - val_loss: 0.1271 - val_accuracy: 0.9540
# Epoch 5/15
# 200/200 - 43s - loss: 0.0303 - accuracy: 0.9910 - val_loss: 0.0384 - val_accuracy: 0.9905
# Epoch 6/15
# 200/200 - 43s - loss: 0.0221 - accuracy: 0.9950 - val_loss: 6.5264 - val_accuracy: 0.5030
# Epoch 7/15
# 200/200 - 43s - loss: 0.0236 - accuracy: 0.9945 - val_loss: 0.4938 - val_accuracy: 0.8365
# Epoch 8/15
# 200/200 - 43s - loss: 0.0316 - accuracy: 0.9920 - val_loss: 6.4351 - val_accuracy: 0.4750
# Epoch 9/15
# 200/200 - 43s - loss: 0.0202 - accuracy: 0.9950 - val_loss: 0.2495 - val_accuracy: 0.8995
# Epoch 10/15
# 200/200 - 43s - loss: 0.0191 - accuracy: 0.9950 - val_loss: 0.4350 - val_accuracy: 0.8280
# Epoch 11/15
# 200/200 - 43s - loss: 0.0162 - accuracy: 0.9965 - val_loss: 0.0129 - val_accuracy: 0.9965
# Epoch 12/15
# 200/200 - 43s - loss: 0.0131 - accuracy: 0.9965 - val_loss: 3.5247 - val_accuracy: 0.5040
# Epoch 13/15
# 200/200 - 43s - loss: 0.0099 - accuracy: 0.9975 - val_loss: 0.5439 - val_accuracy: 0.8000
# Epoch 14/15
# 200/200 - 43s - loss: 0.0141 - accuracy: 0.9950 - val_loss: 0.2691 - val_accuracy: 0.8865
# Epoch 15/15
# 200/200 - 43s - loss: 0.0068 - accuracy: 0.9990 - val_loss: 2.0575 - val_accuracy: 0.5645

In [ ]:
## model3_E15_B10_N2000_val-acc0.983.h5
## CNN + 트랜스포머 결과
## 데이터 2000개만 (메모리오류)

# Epoch 1/15
# 200/200 - 39s - loss: 0.3059 - accuracy: 0.8685 - val_loss: 3.0019 - val_accuracy: 0.4400
# Epoch 2/15
# 200/200 - 25s - loss: 0.1277 - accuracy: 0.9640 - val_loss: 1.1292 - val_accuracy: 0.6025
# Epoch 3/15
# 200/200 - 25s - loss: 0.0928 - accuracy: 0.9685 - val_loss: 0.2415 - val_accuracy: 0.8770
# Epoch 4/15
# 200/200 - 25s - loss: 0.1089 - accuracy: 0.9650 - val_loss: 0.1295 - val_accuracy: 0.9525
# Epoch 5/15
# 200/200 - 25s - loss: 0.0501 - accuracy: 0.9875 - val_loss: 0.2519 - val_accuracy: 0.8940
# Epoch 6/15
# 200/200 - 26s - loss: 0.0878 - accuracy: 0.9715 - val_loss: 0.5996 - val_accuracy: 0.7595
# Epoch 7/15
# 200/200 - 26s - loss: 0.0609 - accuracy: 0.9790 - val_loss: 0.0753 - val_accuracy: 0.9705
# Epoch 8/15
# 200/200 - 26s - loss: 0.0270 - accuracy: 0.9950 - val_loss: 0.0416 - val_accuracy: 0.9865
# Epoch 9/15
# 200/200 - 26s - loss: 0.0467 - accuracy: 0.9845 - val_loss: 0.0839 - val_accuracy: 0.9690
# Epoch 10/15
# 200/200 - 25s - loss: 0.0366 - accuracy: 0.9900 - val_loss: 0.0764 - val_accuracy: 0.9690
# Epoch 11/15
# 200/200 - 25s - loss: 0.0266 - accuracy: 0.9925 - val_loss: 0.1577 - val_accuracy: 0.9400
# Epoch 12/15
# 200/200 - 25s - loss: 0.0238 - accuracy: 0.9940 - val_loss: 0.0344 - val_accuracy: 0.9900
# Epoch 13/15
# 200/200 - 25s - loss: 0.0407 - accuracy: 0.9895 - val_loss: 0.1058 - val_accuracy: 0.9600
# Epoch 14/15
# 200/200 - 25s - loss: 0.0143 - accuracy: 0.9975 - val_loss: 0.1084 - val_accuracy: 0.9610
# Epoch 15/15
# 200/200 - 25s - loss: 0.0108 - accuracy: 0.9980 - val_loss: 0.0619 - val_accuracy: 0.9825


In [ ]:
## model1_E15_B10_N2000_val-acc0.909.h5
## CNN 단일결과
## 데이터 2000개만 

# Epoch 1/15
# 200/200 - 31s - loss: 0.3659 - accuracy: 0.8540 - val_loss: 1.9535 - val_accuracy: 0.4545
# Epoch 2/15
# 200/200 - 17s - loss: 0.1773 - accuracy: 0.9365 - val_loss: 0.9802 - val_accuracy: 0.6025
# Epoch 3/15
# 200/200 - 17s - loss: 0.1207 - accuracy: 0.9605 - val_loss: 0.5959 - val_accuracy: 0.7880
# Epoch 4/15
# 200/200 - 17s - loss: 0.1172 - accuracy: 0.9580 - val_loss: 1.2333 - val_accuracy: 0.6810
# Epoch 5/15
# 200/200 - 17s - loss: 0.0827 - accuracy: 0.9695 - val_loss: 0.0857 - val_accuracy: 0.9720
# Epoch 6/15
# 200/200 - 17s - loss: 0.0643 - accuracy: 0.9780 - val_loss: 0.1298 - val_accuracy: 0.9460
# Epoch 7/15
# 200/200 - 17s - loss: 0.0550 - accuracy: 0.9820 - val_loss: 0.0971 - val_accuracy: 0.9640
# Epoch 8/15
# 200/200 - 17s - loss: 0.0443 - accuracy: 0.9880 - val_loss: 0.0988 - val_accuracy: 0.9570
# Epoch 9/15
# 200/200 - 17s - loss: 0.0332 - accuracy: 0.9905 - val_loss: 0.5075 - val_accuracy: 0.8320
# Epoch 10/15
# 200/200 - 17s - loss: 0.0366 - accuracy: 0.9880 - val_loss: 0.2110 - val_accuracy: 0.9310
# Epoch 11/15
# 200/200 - 17s - loss: 0.0215 - accuracy: 0.9955 - val_loss: 0.1176 - val_accuracy: 0.9510
# Epoch 12/15
# 200/200 - 17s - loss: 0.0411 - accuracy: 0.9870 - val_loss: 0.1250 - val_accuracy: 0.9545
# Epoch 13/15
# 200/200 - 17s - loss: 0.0361 - accuracy: 0.9900 - val_loss: 0.3029 - val_accuracy: 0.8920
# Epoch 14/15
# 200/200 - 17s - loss: 0.0265 - accuracy: 0.9915 - val_loss: 0.2016 - val_accuracy: 0.9325
# Epoch 15/15
# 200/200 - 17s - loss: 0.0316 - accuracy: 0.9895 - val_loss: 0.2543 - val_accuracy: 0.9085